### clone代码

In [ ]:
!git clone https://github.com/modelscope/modelscope-agent.git

### 安装特定依赖

In [ ]:
!cd modelscope-agent && pip install -r requirements.txt
!pip install vllm

### 本地部署llm
modelscope提供模型本地启动服务功能。这里我们使用该功能，将模型部署成openai api兼容的接口。具体操作可参考如下：
#### 下载模型

In [ ]:
!python -c "from modelscope import snapshot_download; model_dir = snapshot_download('qwen/Qwen1.5-7B-Chat', cache_dir='qwen1.5-7b-chat');print(model_dir)"

#### 部署模型

In [ ]:
# 在终端里执行下面的命令启动llm模型服务
!MODELSCOPE_CACHE='demo/qwen1.5-7b-chat' python -m vllm.entrypoints.openai.api_server \
    --model qwen/Qwen1.5-7B-Chat --dtype=half --max-model-len 8192  --gpu-memory-utilization 0.95

#### 测试部署
测试模型服务，如果正确返回，说明模型服务部署完成。

In [ ]:
!curl http://localhost:8000/v1/chat/completions \
    -H "Content-Type: application/json" \
    -d '{
        "model": "qwen/Qwen1.5-7B-Chat",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "写一篇春天为主题的作文"}
        ],
        "stop": ["<|im_end|>", "<|endoftext|>"]
    }'

### 部署agent
#### 拉起agent gradio

In [ ]:
# 另起一个终端窗口，后面将在这个终端里拉起gradio
!cd modelscope-agent/apps/agentfabric
!pip install -r requirements.txt

编辑config/model_config.json， 增加如下配置

In [ ]:
"qwen1.5-7b-chat-vllm-local": {
        "type": "openai",
        "model": "qwen/Qwen1.5-7B-Chat",
        "api_base": "http://localhost:8000/v1",
        "is_chat": true,
        "is_function_call": false
    }

在agentfabric目录下执行如下命令拉起gradio

In [ ]:
!GRADIO_SERVER_NAME=0.0.0.0 PYTHONPATH=../../  python app.py

然后在浏览器中输入 服务器IP:7860打开即可看到如下界面。
![Alt text](../docs/resource/local_deploy.png)